In [2]:
from transformers import AutoTokenizer

In [3]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Inspect FinBERT Vocabulary 

In [38]:
tokenizer = AutoTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

In [5]:
with open("fin_bert_vocabulary.txt","w") as f:
    for token in tokenizer.vocab.keys():
        f.write(token+"\n")

In [6]:
list(tokenizer.vocab.keys())[5000:5020]

['mountains',
 'seventh',
 'slipped',
 'brick',
 '07',
 '##py',
 'somehow',
 'Carter',
 'Iraq',
 'cousin',
 'favor',
 'islands',
 'journey',
 'FIFA',
 'contrast',
 'planet',
 'vs',
 'calm',
 '##ings',
 'concrete']

In [7]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
After         1,258
stealing     11,569
money         1,948
from          1,121
the           1,103
bank          3,085
vault        13,454
,               117
the           1,103
bank          3,085
r               187
##ob         12,809
##ber         3,169
was           1,108
seen          1,562
fishing       5,339
on            1,113
the           1,103
Mississippi   5,201
river         2,186
bank          3,085
.               119
[SEP]           102


# 2.3. Segment ID

In [8]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# # Extracting Embeddings

Running FinBERT on text

In [9]:
import torch
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [10]:
import transformers

config = transformers.AutoConfig.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT", output_hidden_states=True)
model = transformers.AutoModel.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT", config=config)

In [11]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [13]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 24
Number of hidden units: 768


In [14]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 24, 768])


In [15]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 24, 768])

In [16]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 24, 768])

In [17]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([24, 13, 768])

# Creating word and sentence vectors from hidden states


Word Vectors

In [18]:
#First, let's concatenate the last four layers, giving us a single word vector per token. Each vector will have length 4 x 768 = 3,072.

# Stores the token vectors, with shape [24 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [24 x 13 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 24 x 3072


In [19]:
#As an alternative method, let's try creating the word vectors by **summing** together the last four layers.
# Stores the token vectors, with shape [24 x 768]
token_vecs_sum = []

# `token_embeddings` is a [24 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 24 x 768


# Sentence Vectors

In [20]:
# `hidden_states` has shape [13 x 1 x 24 x 768]

# `token_vecs` is a tensor with shape [24 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 24 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [21]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


Confirming contextually dependent vectors

In [22]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 After
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 r
12 ##ob
13 ##ber
14 was
15 seen
16 fishing
17 on
18 the
19 Mississippi
20 river
21 bank
22 .
23 [SEP]


In [23]:
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(token_vecs_sum[6][:5]))
print("bank robber  ", str(token_vecs_sum[10][:5]))
print("river bank   ", str(token_vecs_sum[19][:5]))

First 5 vector values for each instance of "bank".

bank vault    tensor([-3.8447,  2.4839, -3.3137,  2.8949,  3.0209])
bank robber   tensor([-3.2456, -0.8236, -5.3658,  2.9249,  2.8613])
river bank    tensor([ 2.5981, -0.3203, -1.9274, -2.6334,  4.9703])


In [24]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.84
Vector similarity for *different* meanings:  0.47


# Semantic Similarity 

In [25]:
#load sentences from file
with open('FinancialPhraseBank-v1.0/FinancialPhraseBank-v1.0/Sentences_50Agree.txt', encoding="ISO-8859-1") as f:
    sentences = [line.rstrip().split('@')[0] for line in f]

Text to Embedding

In [26]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Token IDs: [101, 1792, 1106, 13529, 117, 1103, 1419, 1144, 1185, 2714, 1106, 1815, 1155, 1707, 1106, 2733, 117, 1780, 1115, 1110, 1187, 1103, 1419, 1110, 2898, 119, 102]


In [27]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  155


In [28]:
import sys
sys.path.append('/home/tamil/anaconda3/envs/dl_env/lib/python3.6/site-packages')
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 160 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 155...
MAX_LEN = 160

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 160 values...

Padding token: "[PAD]", ID: 0

Done.


In [29]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [60]:
device = torch.device('cuda')
model.output_hidden_states = True
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [81]:
def text_to_embedding(tokenizer, model, in_text):
    MAX_LEN = 160

    input_ids = tokenizer.encode(
                        in_text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN
                   )
    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
    
    # Remove the outer list
    input_ids = results[0]
    
    # Create attention masks
    attn_mask = [int(i > 0) for i in input_ids]
    
    #Cast to tensors
    input_ids = torch.tensor(input_ids)
    attn_mask = torch.tensor(attn_mask)
    
    #Add an extra dimension for the "batch" 
    input_ids = input_ids.unsqueeze(0)
    attn_mask = attn_mask.unsqueeze(0)
    
    # Put the model in evaluation mode
    model.eval()
    
    
    # Copy the inputs to GPU
    input_ids = input_ids.to(device)
    attn_mask = attn_mask.to(device)

    
    with torch.no_grad():
        
        # Return hidden states and predictions. This will return the logits rather than loss because we not provided any labels
        encoded_layers = model(input_ids = input_ids, token_type_ids = None, attention_mask = attn_mask)
        
        # Retrieve sentence embedding -- take the '[CLS]' embedding from the final layer
        layer_i = 12 # The last BERT layer before the classifier
        batch_i = 0 # only one input in the bacth
        token_i = 0 # The first token, corresponding to [CLS]
        
        
        # Grab the embedding
        vec = encoded_layers[2][layer_i][batch_i][token_i]
        
        # Move to the CPU and convert it to numpy array
        vec = vec.detach().cpu().numpy()
       
        return(vec)
        

Vectorize dataset

In [82]:
import textwrap
input_text = sentences[10]

wrapper = textwrap.TextWrapper(initial_indent="  ",subsequent_indent=" ",width=80)

print("Getting embedding for the sentence:\n\n", wrapper.fill(input_text))

# Use the BERT model and tokenizer to generate an embedding for input_text
vec = text_to_embedding(tokenizer, model, input_text)

print("\n Done. Embedding shape:", str(vec.shape))

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Getting embedding for the sentence:

   TeliaSonera TLSN said the offer is in line with its strategy to increase its
 ownership in core business holdings and would strengthen Eesti Telekom 's
 offering to its customers .

 Done. Embedding shape: (768,)


In [83]:
import time
import datetime

def format_time(elapsed):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    
    #Round to the nearest second
    elapsed_round = int(round(elapsed))
    
    #Format as hh:mm:ss
    return str(datetime.timedelta(seconds = elapsed_round))

Loop over all sentences

In [86]:
import time

t0 = time.time()

#Store the embeddings
embeddings = []

num_sentences =len(sentences)

print('Generating sentence embeddings for all {:,} sentence...'.format(num_sentences))

row_num = 0

# For each sentence
for sent in sentences:
    
    elapsed = format_time(time.time() - t0)
    
    if row_num % 200 == 0 and not row_num == 0:
        rows_per_sec = (time.time() - t0)/row_num
        remaining_sec = rows_per_sec * (num_sentences - row_num)
        remaining = format_time(remaining_sec)
        
        print(' Sentence {:>7,} of {:>7,}, Elapsed: {:}. Remaining: {:}'. format(row_num, num_sentences, elapsed, remaining))
    
    
    vec = text_to_embedding(tokenizer, model, sent)
    
    # Store the embeddings
    embeddings.append(vec)
    
    row_num += 1
    
    


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Generating sentence embeddings for all 4,846 sentence...


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Sentence   4,800 of   4,846, Elapsed: 0:10:42. Remaining: 0:00:06


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [87]:
import numpy as np

# Convert the list of vectors into a 2D array
vecs = np.stack(embeddings)

vecs.shape

(4846, 768)

In [88]:
import os

output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
#write the embeddings
np.save("./model_save/embeddings.npy",vec)



# Semantic Similarity Search

In [92]:
import faiss

cpu_index = faiss.IndexFlatL2(vecs.shape[1])

n_gpu = 1

print('Number of available GPUs: %d   Using: %d ' %(faiss.get_num_gpus(),n_gpu))

co = faiss.GpuMultipleClonerOptions()
co.shared = True

# Make it into gpu index
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index, co = co, ngpu = n_gpu)

print('Adding dataset to index............')

t0 = time.time()
gpu_index.add(vecs)

elapsed = time.time() - t0

print("Building index took %.2f seconds"%(elapsed)) 

Number of available GPUs: 1   Using: 1 
Adding dataset to index............
Building index took 0.02 seconds


In [99]:
sentences[9]

'Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .'

In [107]:
print("Input Sentence")
print(sentences[20])

# find the 5 most similar sentences

D, I = gpu_index.search(vecs[20].reshape(1,768), k = 5)

print("")
print("Top 5 results")

for i in range(I.shape[1]):
    # look up sentence row number for this result
    results_i = I[0,i]
    
    # look up the text for this sentence
    text = sentences[results_i]
    
    print('Comment #{:,}:'.format(results_i))
    print('L2 Distance: %.2f'%D[0,i])
    print(wrapper.fill('""'+text+'""'))
    print('')

Input Sentence
Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s .

Top 5 results
Comment #20:
L2 Distance: 0.00
  ""Lifetree was founded in 2000 , and its revenues have risen on an average by
 40 % with margins in late 30s .""

Comment #216:
L2 Distance: 3.35
  ""In the fourth quarter of 2009 , Orion 's net profit went up by 33.8 % year-
 on-year to EUR33m .""

Comment #40:
L2 Distance: 3.50
  ""In January-September 2010 , Fiskars ' net profit went up by 14 % year-on-
 year to EUR 65.4 million and net sales to EUR 525.3 million from EUR 487.7
 million .""

Comment #232:
L2 Distance: 3.76
  ""Metso expects its net sales to increase by about 10 % in 2008 , at
 comparable exchange rates .""

Comment #342:
L2 Distance: 3.77
  ""In the reporting period , the company 's operating profit grew by 43.2 % to
 EUR 6 million .""

